In [1]:
import pandas as pd
import numpy as np
import json
import os
import sys
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import plotly.io as pio
pio.renderers.default = "notebook"

path_root = "D:/Documentos/Erik/TDR/TDR-Forex/"
sys.path.append(path_root)
from utils.strategies import load_strategy_parameters, get_strategy, preparing_data_backtest


### Paràmetres fixes de les estratègies

Els paràmetres fixos de les estratègies es defineixen en un fitxer `.json`. Cada estratègia té un identificador únic i pot tenir configuracions diferents per a cada símbol i timeframe.

#### Significat dels principals paràmetres

- **Name**: Nom descriptiu de l'estratègia.
- **MagigNumber**: Identificador numèric de l'estratègia.  
- **Description**: Breu descripció de l'estratègia.  
- **pairs**: Diccionari amb els símbols (parells de divises) als quals s'aplica l'estratègia.  
- **TimeFrame** (clau dins de cada símbol): Finestra temporal (ex: M1, M15, H1, D1) amb configuració específica.  
- **NBars**: Nombre de barres històriques a utilitzar per al càlcul d'indicadors.  
- **DynamicLotSize**: Si `true`, el lot es calcula en funció del risc; si `false`, s’utilitza `FixedLotSize`.  
- **EquityPercent**: Percentatge del capital a arriscar per operació (només si `DynamicLotSize` és `true`).  
- **FixedLotSize**: Mida fixa del lot (només si `DynamicLotSize` és `false`).  
- **MaxOpenTrades**: Nombre màxim de posicions obertes simultàniament per a aquest símbol i timeframe.  
- **MaxMinutesOpenTrades**: Temps màxim (en minuts) que una operació pot estar oberta.  
- **MinBetweenTrades**: Temps mínim (en minuts) entre operacions consecutives.  
- **TP_short / TP_long**: Take Profit per a posicions curtes i llargues (en pips).  
- **SL_short / SL_long**: Stop Loss per a posicions curtes i llargues (en pips).  

In [2]:
filename = os.path.join(path_root, "input/Forex/strategies/dict_strategies.json")
with open(filename, "r", encoding="utf-8") as f:
    dict_strategies = json.load(f)

# Estratègia model
strategy_name = "Mitjanes"
strategy = dict_strategies[strategy_name]
print(f"Estratègia: {strategy_name}")
print(json.dumps(strategy, indent=4, ensure_ascii=False))

Estratègia: Mitjanes
{
    "MagicNumber": 1,
    "Description": "",
    "pairs": {
        "EURUSD": {
            "M1": {
                "NBars": 200,
                "DynamicLotSize": false,
                "EquityPercent": 2.5,
                "FixedLotSize": 0.01,
                "MaxOpenTrades": 5,
                "MaxMinutesOpenTrades": 720,
                "MinBetweenTrades": 5,
                "TP_short": 10,
                "TP_long": 10,
                "SL_long": 25,
                "SL_short": 25
            },
            "H1": {
                "NBars": 200,
                "DynamicLotSize": false,
                "EquityPercent": 2.5,
                "FixedLotSize": 0.01,
                "MaxOpenTrades": 5,
                "MaxMinutesOpenTrades": 720,
                "MinBetweenTrades": 5,
                "TP_short": 10,
                "TP_long": 10,
                "SL_long": 25,
                "SL_short": 25
            }
        }
    }
}


### Funció `load_strategy_parameters`

Carrega els paràmetres d'una estratègia des d'un fitxer JSON i retorna un diccionari pla combinant els paràmetres generals amb els específics per a un `symbol` i `timeframe` donats, llest per generar senyals.

In [3]:
strategy_name = "Mitjanes"; symbol = "EURUSD"; timeframe = "H1"; realtime = True
strategy_params = load_strategy_parameters(strategy_name, symbol, timeframe, path_root)
print("PARÀMETRES DEL SISTEMA:")
print(json.dumps(strategy_params, indent=2, ensure_ascii=False))

PARÀMETRES DEL SISTEMA:
{
  "MagicNumber": 1,
  "Description": "",
  "Symbol": "EURUSD",
  "TimeFrame": "H1",
  "NBars": 200,
  "DynamicLotSize": false,
  "EquityPercent": 2.5,
  "FixedLotSize": 0.01,
  "MaxOpenTrades": 5,
  "MaxMinutesOpenTrades": 720,
  "MinBetweenTrades": 5,
  "TP_short": 10,
  "TP_long": 10,
  "SL_long": 25,
  "SL_short": 25
}


### Funció `load_strategy_parameters` amb `realtime=True`

Aquesta funció carrega els paràmetres d'una estratègia des d'un fitxer JSON i els combina amb els valors específics per a un símbol i un timeframe. Quan s'utilitza amb `realtime=True`, els paràmetres carregats s'apliquen per obtenir dades en temps real des de MetaTrader 5, mantenint-los llestos per generar senyals immediats.  

**Paràmetres d'entrada:**

- `strategy_name` (str): Nom de l'estratègia, clau principal del JSON.  
- `symbol` (str): Parell de divises per al qual es volen carregar els paràmetres (ex: 'EURUSD').  
- `timeframe` (str): Marc temporal per al qual es volen carregar els paràmetres (ex: 'M1', 'H1').  

**Retorn:**

- Diccionari amb tots els paràmetres combinats, incloent:  
  - `MagicNumber` i `Description` generals de l'estratègia.  
  - `Symbol` i `TimeFrame` seleccionats.  
  - Paràmetres específics del símbol i timeframe, com `NBars`, `DynamicLotSize`, `EquityPercent`, `FixedLotSize`, `MaxOpenTrades`, `TP_long`, `SL_short`, etc.  

**Funcionament resumit:**

1. Obre i llegeix el fitxer JSON amb tots els paràmetres de les estratègies.  
2. Recupera els paràmetres generals de l'estratègia seleccionada.  
3. Combina aquests paràmetres amb els valors específics per al símbol i timeframe indicats.  
4. Retorna un diccionari pla que es pot utilitzar per obtenir dades en temps real i generar senyals immediats.  

In [4]:
# Obtenir dades i senyals de l'estratègia
data = get_strategy(strategy_name, symbol, timeframe, path_root, realtime=True)
data.tail()

,time,signal,cond_close_long,cond_close_short,open,high,low,close,ema18,ema30,ema200
0,2025-08-29 13:00:00,0.0,0.0,0.0,1.16725,1.16739,1.16646,1.16675,1.16691,1.166477,1.165217


### Funció `get_strategy` amb `realtime=False`

Aquesta funció carrega dades històriques d'un símbol i timeframe donats des de fitxers locals i les prepara per a simulacions (backtest). Genera senyals d'entrada i sortida segons l'estratègia seleccionada, calcula indicadors necessaris (ex: EMAs) i retorna un DataFrame complet amb totes les barres i senyals calculats.  

**Paràmetres d'entrada:**

- `strategy_name` (str): Nom de l'estratègia (ex: 'Mitjanes').  
- `symbol` (str): Símbol del parell de divises (ex: 'EURUSD').  
- `timeframe` (str): Marc temporal per carregar les dades (ex: 'M1', 'H1').  
- `path_root` (str): Ruta arrel on es troben els fitxers de dades i configuració.  
- `realtime` (bool, opcional): Quan és `False` utilitza dades històriques per backtest.  

**Retorn:**

- `DataFrame`: Conté totes les barres del timeframe amb:  
  - Columnes d'indicadors calculats (ex: `ema18`, `ema30`, `ema200`).  
  - Senyals generats (`signal`, `cond_close_long`, `cond_close_short`).  
  - Informació temporal (`time`).  

**Funcionament resumit:**

1. Carrega els paràmetres de l'estratègia mitjançant `load_strategy_parameters`.  
2. Obté les dades històriques del símbol i timeframe indicats.  
3. Calcula els indicadors necessaris i genera els senyals d'entrada i sortida segons l'estratègia.  
4. Retarda els senyals una barra per evitar forward-looking bias.  
5. Retorna un DataFrame llest per a l'anàlisi o backtesting.  


In [5]:
# Obtenir dades i senyals de l'estratègia
data = get_strategy(strategy_name, symbol, timeframe, path_root, realtime=False)
display(data[data.signal==1].head(1))
display(data[(data.time>=pd.to_datetime("2000-01-11 09:00:00")) & (data.time<=pd.to_datetime("2000-01-11 13:00:00"))])

Ordres de compra: 2074
Ordres de venda: 2074
Ratio compra/venda: 1.00


,time,signal,cond_close_long,cond_close_short,open,high,low,close,ema18,ema30,ema200
155,2000-01-11 11:00:00,1.0,0.0,0.0,1.0296,1.0318,1.0288,1.0301,1.027654,1.027448,1.024441


,time,signal,cond_close_long,cond_close_short,open,high,low,close,ema18,ema30,ema200
153,2000-01-11 09:00:00,0.0,0.0,0.0,1.0283,1.0312,1.0280,1.0312,1.027103,1.027104,1.024332
154,2000-01-11 10:00:00,0.0,0.0,0.0,1.0309,1.0312,1.0288,1.0296,1.027366,1.027265,1.024384
155,2000-01-11 11:00:00,1.0,0.0,0.0,1.0296,1.0318,1.0288,1.0301,1.027654,1.027448,1.024441
156,2000-01-11 12:00:00,0.0,0.0,0.0,1.0299,1.0308,1.0284,1.0292,1.027817,1.027561,1.024489
157,2000-01-11 13:00:00,0.0,0.0,0.0,1.0294,1.0294,1.0283,1.0289,1.027931,1.027647,1.024533


En el DataFrame de l'estratègia apareixen tres columnes de senyals principals:

- `signal`: Indica la direcció d'entrada a mercat segons l'estratègia.  
  - `1.0` → senyal de compra (long).  
  - `-1.0` → senyal de venda (short).  
  - `0.0` → cap senyal d'entrada en aquesta barra.

- `cond_close_long`: Senyal de tancament d'una posició long oberta.  
  - `1.0` → condició de tancament complerta (tancar posició long).  
  - `0.0` → mantenir la posició oberta o no hi ha posició.

- `cond_close_short`: Senyal de tancament d'una posició short oberta.  
  - `1.0` → condició de tancament complerta (tancar posició short).  
  - `0.0` → mantenir la posició oberta o no hi ha posició.

Aquestes columnes permeten reconstruir l'execució de l'estratègia: `signal` genera les ordres d'entrada, i `cond_close_long` / `cond_close_short` determinen quan cal tancar-les. En combinació amb els indicadors (`ema18`, `ema30`, `ema200`, etc.), es pot analitzar l'eficàcia de l'estratègia.


### Funció `preparing_data_backtest`

Aquesta funció prepara les dades per a un backtest quan l'estratègia s'ha calculat en un timeframe diferent de M1. L'objectiu és propagar les senyals generades al timeframe original a les barres de 1 minut corresponents.

**Com funciona:**

1. Crea una columna auxiliar `time_hour` que redueix la marca temporal a l'hora exacta (sense minuts) per fer el merge amb les dades M1.
2. Carrega les dades històriques en timeframe M1 del símbol corresponent.
3. Fa un merge de les senyals del timeframe original amb les barres M1 de la mateixa hora.
4. Assigna les senyals només a la primera barra M1 de cada hora (`signal`, `cond_close_long`, `cond_close_short`), mantenint 0 en les altres barres.
5. Neteja columnes auxiliars i retorna un DataFrame en timeframe M1, llest per al backtest detallat minut a minut.

Aquesta aproximació assegura que les senyals calculades en timeframes més grans es poden simular amb precisió en dades M1, evitant duplicats i errors en la propagació de senyals.


In [6]:
data = preparing_data_backtest(data, symbol, timeframe, path_root)
data[data.time >= pd.to_datetime("2000-01-11 11:00:00")].head()

,time,open,high,low,close,tick_volume,signal,cond_close_long,cond_close_short
8221,2000-01-11 11:00:00,1.0296,1.0301,1.0294,1.0301,8,1.0,0.0,0.0
8222,2000-01-11 11:01:00,1.0296,1.0298,1.0295,1.0295,5,0.0,0.0,0.0
8223,2000-01-11 11:02:00,1.0296,1.0297,1.0292,1.0293,6,0.0,0.0,0.0
8224,2000-01-11 11:03:00,1.0292,1.0296,1.0289,1.0291,11,0.0,0.0,0.0
8225,2000-01-11 11:04:00,1.0293,1.0297,1.0288,1.0291,12,0.0,0.0,0.0


### Procés per a temps real

En mode temps real (`realtime=True`), la funció `get_strategy`:

1. Inicialitza la connexió amb MetaTrader 5 i comprova que estigui activa.
2. Obtén les darreres dades històriques per al `symbol` i `timeframe` indicats, limitant el nombre de barres segons `NBars`.
3. Converteix les dades obtingudes a un DataFrame i elimina columnes no necessàries (`real_volume`, `spread`).
4. Calcula els indicadors necessaris (per exemple, EMAs, SMA, etc.) segons l'estratègia.
5. Genera els senyals d'entrada i condicions de tancament per l'estratègia seleccionada.
6. Retarda els senyals una barra per evitar utilitzar informació futura.
7. Retorna només l'última barra amb tots els indicadors i senyals, llesta per executar operacions en temps real.

In [7]:
strategy_name = "Mitjanes"; symbol = "EURUSD"; timeframe = "M1"; realtime = True
strategy_params = load_strategy_parameters(strategy_name, symbol, timeframe, path_root)
data = get_strategy(strategy_name, symbol, timeframe, path_root, realtime)
data

,time,signal,cond_close_long,cond_close_short,open,high,low,close,ema18,ema30,ema200
0,2025-08-29 14:07:00,0.0,0.0,0.0,1.16658,1.16661,1.1665,1.1665,1.166654,1.166726,1.167079


### Procés per a backtest

En mode backtest (`realtime=False`), el procés és el següent:

1. Carrega les dades històriques del `symbol` i `timeframe` seleccionats des dels fitxers locals.
2. Calcula els indicadors necessaris (EMAs, SMA, etc.) i genera els senyals d’entrada i condicions de tancament per a l’estratègia.
3. Retarda els senyals una barra per evitar utilitzar informació futura.
4. Si el timeframe no és M1, la funció `preparing_data_backtest` propaga les senyals del timeframe original a les barres M1 corresponents:
   - Les senyals es mantenen només a la primera barra M1 de cada període del timeframe original.
   - Les altres barres dins del mateix període tenen senyals neutres (0).
5. Retorna un DataFrame complet amb totes les barres, indicadors i senyals, llest per a simulacions de trading i càlculs de rendibilitat.

In [8]:
strategy_name = "Mitjanes"; symbol = "EURUSD"; timeframe = "M1"; realtime = False
strategy_params = load_strategy_parameters(strategy_name, symbol, timeframe, path_root)
data = get_strategy(strategy_name, symbol, timeframe, path_root, realtime)
data = preparing_data_backtest(data, symbol, timeframe, path_root)
data.tail()

Ordres de compra: 139859
Ordres de venda: 139858
Ratio compra/venda: 1.00


,time,signal,cond_close_long,cond_close_short,open,high,low,close,ema18,ema30,ema200
9341178,2025-08-18 13:39:00,0.0,0.0,0.0,1.16793,1.16799,1.16791,1.16796,1.167923,1.167905,1.168328
9341179,2025-08-18 13:40:00,0.0,0.0,0.0,1.16795,1.16808,1.16793,1.16803,1.167934,1.167913,1.168326
9341180,2025-08-18 13:41:00,0.0,0.0,0.0,1.16803,1.16805,1.16799,1.16805,1.167946,1.167922,1.168323
9341181,2025-08-18 13:42:00,0.0,0.0,0.0,1.16804,1.16808,1.16802,1.16808,1.167961,1.167932,1.168320
9341182,2025-08-18 13:43:00,0.0,0.0,0.0,1.16808,1.16814,1.16800,1.16814,1.167979,1.167945,1.168319
